In [1]:
import numpy as np
import pandas as pd

In [2]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])

In [3]:
s

a   -1.366551
b    1.325699
c    0.913021
d    0.180595
e   -2.344663
dtype: float64

In [ ]:
s+s

In [ ]:
df = pd.read_csv('actors_Zene3.csv', sep=',')
df.Position = df.Position.astype(float)
df.Capability = df.Capability.astype(float)
df.Salience = df.Salience/100.0
df.head()

In [ ]:
# Location of file
Location = r'c:\Dropbox\Game Theory\Matlab\BDM\Zenekar.xlsx'

# Parse a specific sheet
#df = pd.read_excel(Location, 1, index_col='StatusDate')
df = pd.read_excel(Location, 3)
df.dtypes

In [ ]:
df

In [4]:
df = pd.read_excel(r'c:\Dropbox\Game Theory\Matlab\BDM\Zenekar.xlsx', 3)
df.dtypes

Actor          object
Capability    float64
Position        int64
Salience      float64
dtype: object

In [5]:
df

,Actor,Capability,Position,Salience
0,Peti,0.15,40,0.4
1,Timi,0.10,0,0.9
2,Balazs,0.20,100,0.9
3,Miri,0.05,50,0.2
4,Bence,0.15,60,0.8
5,Viki,0.10,60,0.4
6,Krisz,0.15,20,0.9
7,Krisz Barátnője,0.10,20,0.9


In [13]:
import pandas as pd
import numpy as np
import itertools

Q = 1.0
T = 1.0

class Game(object):
    def __init__(self,df):
        self.df = df                                                 # df = dataframe
        self.df_capability = df.Capability.to_dict()    
        self.df_position = df.Position.to_dict()    
        self.df_salience = df.Salience.to_dict()    
        self.max_pos = df.Position.max()
        self.min_pos = df.Position.min()

    def weighted_median(self):                                       # Median Voter Calculation
        self.df['w'] = self.df.Capability*self.df.Salience           # power 
        self.df['w'] = self.df['w'] / self.df['w'].sum()
        self.df['w'] = self.df['w'].cumsum()
        return float(self.df[self.df['w']>=0.5].head(1).Position)

    def mean(self):
        return (self.df.Capability*self.df.Position*self.df.Salience).sum() / \
               (self.df.Capability*self.df.Salience).sum()

    def Usi_i(self,i,j,ri=1.):                                       # Utility of Success
        tmp1 = self.df_position[i]-self.df_position[j]
        tmp2 = self.max_pos-self.min_pos
        return 2. - 4.0 * ( (0.5-0.5*np.abs(float(tmp1)/tmp2) )**ri)

    def Ufi_i(self,i,j,ri=1.):                                       # Utility of Failure
        tmp1 = self.df_position[i]-self.df_position[j]
        tmp2 = self.df.Position.max()-self.df.Position.min()
        return 2. - 4.0 * ( (0.5+0.5*np.abs(float(tmp1)/tmp2) )**ri )

    def Usq_i(self,i,ri=1.):                                         # Utility of Status Quo
        return 2.-(4.*(0.5**ri))

    def Ui_ij(self,i,j):                                             # ez mi?
        tmp1 = self.df_position[i] - self.df_position[j]
        tmp2 = self.max_pos-self.min_pos
        return 1. - 2.*np.abs(float(tmp1) / tmp2) 

    def v(self,i,j,k):
        return self.df_capability[i]*self.df_salience[i]*(self.Ui_ij(i,j)-self.Ui_ij(i,k)) 

    def Pi(self,i):
        l = np.array([[i,j,k] for (j,k) in itertools.combinations(range(len(self.df)), 2 ) if i!=j and i!=k])
        #U_filter = np.array(map(lambda i,j,k: self.Ui_ij(j,i)>self.Ui_ij(i,k), l))
        U_filter = [self.Ui_ij(j,i)>self.Ui_ij(i,k) for i, j, k in l]
        lpos = l[U_filter]
        #tmp1 = np.sum(map(lambda i,j,k: self.v(j,i,k), lpos))
        #tmp2 = np.sum(map(lambda i,j,k: self.v(j,i,k), l))
        tmp1 = np.sum([self.v(j,i,k) for i, j, k in lpos])
        tmp2 = np.sum([self.v(j,i,k) for i, j, k in l])
        #print('P: ',i,float(tmp1)/tmp2)
        return float(tmp1)/tmp2

    def Ubi_i(self,i,j,ri=1):                                                 # Utility of Changing for Better
        tmp1 = np.abs(self.df_position[i] - self.weighted_median()) + \
               np.abs(self.df_position[i] - self.df_position[j])
        tmp2 = np.abs(self.max_pos-self.min_pos)
        return 2. - (4. * (0.5 - (0.25 * float(tmp1) / tmp2))**ri)

    def Uwi_i(self,i,j,ri=1):                                                 # Utility of Changing for Worse
        tmp1 = np.abs(self.df_position[i] - self.weighted_median()) + \
               np.abs(self.df_position[i] - self.df_position[j])
        tmp2 = np.abs(self.max_pos-self.min_pos)
        return 2. - (4. * (0.5 + (0.25 * float(tmp1) / tmp2))**ri)

    def EU_i(self,i,j,r=1):
        term1 = self.df_salience[j] * \
                ( self.Pi(i)*self.Usi_i(i,j,r) + ( 1.-self.Pi(i) )*self.Ufi_i(i,j,r) )
        term2 = (1-self.df_salience[j])*self.Usi_i(i,j,r)
        #term3 = -self.Qij(j,i)*self.Usq_i(i,r)
        #term4 = -(1.-self.Qij(j,i))*( T*self.Ubi_i(i,j,r) + (1.-T)*self.Uwi_i(i,j,r) )
        term3 = -Q*self.Usq_i(i,r)
        term4 = -(1.-Q)*( T*self.Ubi_i(i,j,r) + (1.-T)*self.Uwi_i(i,j,r) )
        return term1+term2+term3+term4

    def EU_j(self,i,j,r=1):
        return self.EU_i(j,i,r)

    def Ri(self,i):
        # get all j's except i
        l = [x for x in range(len(self.df)) if x!= i]
        #tmp = np.array(list(map(lambda x: self.EU_j(i,x), l)))
        tmp = np.array([self.EU_j(i,x) for x in l])
        numterm1 = 2 * np.sum(tmp)
        numterm2 = (len(self.df)-1)*np.max(tmp)
        numterm3 = (len(self.df)-1)*np.min(tmp)
        return float(numterm1-numterm2-numterm3) / (numterm2-numterm3)

    def ri(self,i):
        Ri_tmp = self.Ri(i)
        return (1-Ri_tmp/3.) / (1+Ri_tmp/3.)

    def Qij(self,i,j):
        l = np.array([k for k in range(len(self.df))])
        res = map(lambda x: self.Pi(k)+(1-self.df_salience[k]),l)     #To convert all the State values to upper case we will use the upper() function and the dataframe's apply attribute. The lambda function simply will apply the upper function to each value in the State column.
        return np.product(res)

    def calc_offer(self, x):
        if len(x)>0:
            return x[np.argmin(np.abs(x))]
        return 0

    def do_round(self,df):
        self.df = df; df_new = self.df.copy()        
        # reinit
        self.df_capability = self.df.Capability.to_dict()    
        self.df_position = self.df.Position.to_dict()    
        self.df_salience = self.df.Salience.to_dict()    
        self.max_pos = self.df.Position.max()
        self.min_pos = self.df.Position.min()

        offers = [list() for i in range(len(self.df))]
        ris = [self.ri(i) for i in range(len(self.df))]
        for (i,j) in itertools.combinations(range(len(self.df)), 2 ):           # combinations('ABCD', 2) --> AB AC AD BC BD CD
            eui = self.EU_i(i,j,r=ris[i])
            euj = self.EU_j(i,j,r=ris[j])
            if eui > 0 and euj > 0:
                # conflict
                mid_step = (self.df_position[i]-self.df_position[j])/2.
                print(i,j,eui,euj,'conflict, both step', mid_step, -mid_step)
                offers[j].append(mid_step)                                      # append a new item with value x to the end of the array.
                offers[i].append(-mid_step)
            elif eui > 0 and euj < 0 and np.abs(eui) > np.abs(euj):
                # compromise - actor i has the upper hand
                print(i,j,eui,euj,'compromise', i, 'upper hand')
                xhat = (self.df_position[i]-self.df_position[j]) * np.abs(euj/eui)
                offers[j].append(xhat)
            elif eui < 0 and euj > 0 and np.abs(eui) < np.abs(euj):
                # compromise - actor j has the upper hand
                print(i,j,eui,euj,'compromise', j, 'upper hand')
                xhat = (self.df_position[j]-self.df_position[i]) * np.abs(eui/euj)
                offers[i].append(xhat)
            elif eui > 0 and euj < 0 and np.abs(eui) < np.abs(euj):
                # capinulation - actor i has upper hand
                j_moves = self.df_position[i]-self.df_position[j]
                print(i,j,eui,euj,'capitulate', i, 'wins', j, 'moves',j_moves)
                offers[j].append(j_moves)
            elif eui < 0 and euj > 0 and np.abs(eui) > np.abs(euj):
                # capitulation - actor j has upper hand
                i_moves = self.df_position[j]-self.df_position[i]
                print(i,j,eui,euj,'capitulate', j, 'wins', i, 'moves',i_moves)
                offers[i].append(i_moves)
            else:
                print(i,j,eui,euj,'nothing')

In [15]:
print(EU_i(df,1,2,1))

AttributeError: 'DataFrame' object has no attribute 'df_salience'